In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [4]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [5]:
lrModel = LogisticRegression()

In [6]:
fittedModel = lrModel.fit(my_data)

In [7]:
summary = fittedModel.summary

In [10]:
summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [11]:
lrTrain, lrTest = my_data.randomSplit([0.7,0.3])

In [12]:
fittedModel = lrModel.fit(lrTrain)

In [13]:
results = fittedModel.evaluate(lrTest)

In [14]:
results.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[20.4818920657068...|[0.99999999872700...|       0.0|
|  0.0|(692,[100,101,102...|[9.85811584106541...|[0.99994768189969...|       0.0|
|  0.0|(692,[121,122,123...|[20.7085513314935...|[0.99999999898517...|       0.0|
|  0.0|(692,[122,123,148...|[20.8837309996043...|[0.99999999914825...|       0.0|
|  0.0|(692,[123,124,125...|[18.0319781164762...|[0.99999998524934...|       0.0|
|  0.0|(692,[126,127,128...|[28.1796097060215...|[0.99999999999942...|       0.0|
|  0.0|(692,[126,127,128...|[20.9281593036845...|[0.99999999918526...|       0.0|
|  0.0|(692,[127,128,129...|[19.6316793923327...|[0.99999999702100...|       0.0|
|  0.0|(692,[128,129,130...|[21.8089408688898...|[0.99999999966232...|       0.0|
|  0.0|(692,[150

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [16]:
my_eval = BinaryClassificationEvaluator()

In [17]:
myRoc = my_eval.evaluate(results.predictions)

In [18]:
myRoc

0.9963235294117647

In [19]:
df = spark.read.csv('titanic.csv',inferSchema=True,header=True)

In [20]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [21]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [22]:
my_cols = df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [23]:
my_cols.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



In [24]:
my_final_data = my_cols.na.drop()

In [25]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer,OneHotEncoder,StringIndexer)

In [26]:
Gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')

In [27]:
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [28]:
embarked_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embarked_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [29]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],outputCol='features')

In [33]:
from pyspark.ml import Pipeline

In [34]:
titanicLR = LogisticRegression(featuresCol='features',labelCol='Survived')

In [35]:
pipeline = Pipeline(stages = [Gender_indexer,embarked_indexer,
                              gender_encoder,embarked_encoder,
                              assembler,titanicLR])

In [36]:
trainData, testData = my_final_data.randomSplit([0.7,0.3])

In [37]:
fitModel = pipeline.fit(trainData)

In [38]:
results = fitModel.transform(testData) 
#Automatically called its prediction column as prediction, which can be passed to BinaryClassificationEvaluator

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [41]:
myEval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [42]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
+--------+----------+
only showing top 20 rows



In [43]:
AUC = myEval.evaluate(results)

In [45]:
AUC

0.7661238825031929

In [48]:
df = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [55]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [52]:
df

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)]

In [56]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

Name: Name of the latest contact at company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
Account_Manager: Binary 0=No manager, 1 = Account manager assinged
Years: Totaly Years as a customer 
Num_Sites: Number of websites that use the service.
Onboard_date: Date that the name of the latest contact was onboarded
Location: HQ address
Company: Name of the client Company

In [60]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [62]:
assembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'],outputCol='features')

In [63]:
output = assembler.transform(df)

In [64]:
output.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn',
 'features']

In [66]:
final_data = output.select('Churn','features')

In [68]:
trainChurn,testChrun = final_data.randomSplit([0.7,0.3])

In [69]:
lr_chrun = LogisticRegression(labelCol='Churn')

In [71]:
churnModel = lr_chrun.fit(trainChurn)

In [72]:
results = churnModel.evaluate(testChrun)

In [79]:
a = results.

In [74]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [75]:
auc = churn_eval.evaluate(results.predictions)

In [95]:
type(auc)

float

### Predict on new data

In [84]:
final_lr_model = lr_chrun.fit(final_data)

In [85]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [90]:
test_new_customer.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [88]:
test_new_customer = assembler.transform(new_customers)

In [92]:
final_results = final_lr_model.transform(test_new_customer)

In [93]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.22168705251440...|[0.90218018099704...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595020...|[0.00198380445828...|       

In [97]:
final_results.select('Company','rawPrediction','probability','prediction').show()

+----------------+--------------------+--------------------+----------+
|         Company|       rawPrediction|         probability|prediction|
+----------------+--------------------+--------------------+----------+
|        King Ltd|[2.22168705251440...|[0.90218018099704...|       0.0|
|   Cannon-Benson|[-6.2207530595020...|[0.00198380445828...|       1.0|
|Barron-Robertson|[-3.7691621189402...|[0.02255110110413...|       1.0|
|   Sexton-Golden|[-5.0956222016519...|[0.00608622642084...|       1.0|
|        Wood LLC|[1.10475867224153...|[0.75115067517475...|       0.0|
|   Parks-Robbins|[-1.6896019277064...|[0.15582819767635...|       1.0|
+----------------+--------------------+--------------------+----------+

